Raster CLI and API sidebyside

In [1]:
import os
os.chdir(r"D:\1. PROJECT\satellite_etl\satellite\reference image")
os.getcwd()

'D:\\1. PROJECT\\satellite_etl\\satellite\\reference image'

# Getting RGB and NIR in the satellite data

### CLI 

In [ ]:
!gdalinfo oldimage.tif

In [18]:
!gdal_translate -b 1 -b 2 -b 3 oldimage.tif rgb.tif

Input file size is 2550, 2809
0...10...20...30...40...50...60...70...80...90...100 - done.


In [20]:
!gdal_translate -b 4 oldimage.tif nir.tif

Input file size is 2550, 2809
0...10...20...30...40...50...60...70...80...90...100 - done.


### Using osgeo_gdal

In [21]:
from osgeo import gdal

infile = "oldimage.tif"
ds = gdal.Open(infile)

driver = gdal.GetDriverByName("GTiff")
rgb_ds = driver.Create("rgb_python.tif", ds.RasterXSize, ds.RasterYSize, 3, gdal.GDT_UInt16)
rgb_ds.SetProjection(ds.GetProjection())
rgb_ds.SetGeoTransform(ds.GetGeoTransform())
rgb_ds.GetRasterBand(1).WriteArray(ds.GetRasterBand(1).ReadAsArray())  # Red
rgb_ds.GetRasterBand(2).WriteArray(ds.GetRasterBand(2).ReadAsArray())  # Green
rgb_ds.GetRasterBand(3).WriteArray(ds.GetRasterBand(3).ReadAsArray())  # Blue
rgb_ds = None  # close

# --- Save NIR ---
nir_ds = driver.Create("nir_python.tif", ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_UInt16)
nir_ds.SetProjection(ds.GetProjection())
nir_ds.SetGeoTransform(ds.GetGeoTransform())
nir_ds.GetRasterBand(1).WriteArray(ds.GetRasterBand(4).ReadAsArray())  # NIR
nir_ds = None  # close


### Using Rasterio

In [ ]:
import rasterio
import numpy as np

infile = "oldimage.tif"

with rasterio.open(infile) as src:
    profile = src.profile

    # Read bands
    red   = src.read(1)
    green = src.read(2)
    blue  = src.read(3)
    nir   = src.read(4)

    # --- Save RGB ---
    rgb_profile = profile.copy()
    rgb_profile.update(count=3)  # 3-band output
    with rasterio.open("rgb.tif", "w", **rgb_profile) as dst:
        dst.write(red, 1)
        dst.write(green, 2)
        dst.write(blue, 3)

    # --- Save NIR ---
    nir_profile = profile.copy()
    nir_profile.update(count=1)  # single-band output
    with rasterio.open("nir.tif", "w", **nir_profile) as dst:
        dst.write(nir, 1)
